# Downloading models

In [1]:
import inseq
# inseq.list_feature_attribution_methods()
from peft import LoraConfig, PeftModel
from transformers import AutoModelForCausalLM

# loading inseq models
peft_model_id_RL = "DanielSc4/RedPajama-INCITE-Chat-3B-v1-RL-LoRA-8bit-test1"
config_RL = LoraConfig.from_pretrained(peft_model_id_RL)
original_pretrained = AutoModelForCausalLM.from_pretrained(config_RL.base_model_name_or_path)
peft_model_RL = PeftModel(model = original_pretrained, peft_config=config_RL)
merged_model_RL = peft_model_RL.merge_and_unload()

# peft_model_id_FT = "DanielSc4/RedPajama-INCITE-Chat-3B-v1-FT-LoRA-8bit-test1"
# config_FT = LoraConfig.from_pretrained(peft_model_id_FT)
# original_pretrained = AutoModelForCausalLM.from_pretrained(config_FT.base_model_name_or_path)
# peft_model_FT = PeftModel(model = original_pretrained, peft_config=config_FT)
# merged_model_FT = peft_model_FT.merge_and_unload()

original_pretrained = AutoModelForCausalLM.from_pretrained(config_RL.base_model_name_or_path)

/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary /Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
dlopen(/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so, 0x0006): tried: '/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (no such

/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
inseq.list_feature_attribution_methods()

['saliency',
 'occlusion',
 'input_x_gradient',
 'attention',
 'integrated_gradients',
 'discretized_integrated_gradients',
 'layer_deeplift',
 'lime',
 'gradient_shap',
 'layer_gradient_x_activation',
 'deeplift',
 'layer_integrated_gradients']

In [3]:
seq_PT = inseq.load_model(
    original_pretrained,
    attribution_method='input_x_gradient',
)
seq_RL = inseq.load_model(
    merged_model_RL,
    attribution_method='input_x_gradient',
)
# seq_FT = inseq.load_model(
#     merged_model_FT,
#     attribution_method='input_x_gradient',
# )

# Comparison

In [30]:
list(original_pretrained.gpt_neox.layers[30].attention.query_key_value.parameters())

[Parameter containing:
 tensor([[-0.0005,  0.0388, -0.0226,  ...,  0.0320,  0.0112,  0.0160],
         [ 0.0023,  0.0283,  0.0187,  ..., -0.0193,  0.0355,  0.0056],
         [ 0.0246,  0.0122, -0.0097,  ..., -0.0075, -0.0040, -0.0090],
         ...,
         [-0.0206, -0.0134, -0.0168,  ...,  0.0098, -0.0210,  0.0064],
         [-0.0155,  0.0357,  0.0197,  ..., -0.0324, -0.0013,  0.0121],
         [-0.0301, -0.0105,  0.0022,  ..., -0.0111,  0.0353, -0.0132]]),
 Parameter containing:
 tensor([-0.0227, -0.0310,  0.0207,  ...,  0.0025,  0.0026, -0.0005])]

In [31]:
list(merged_model_RL.model.gpt_neox.layers[30].attention.query_key_value.parameters())

[Parameter containing:
 tensor([[-0.0005,  0.0388, -0.0226,  ...,  0.0320,  0.0112,  0.0160],
         [ 0.0023,  0.0283,  0.0187,  ..., -0.0193,  0.0355,  0.0056],
         [ 0.0246,  0.0122, -0.0097,  ..., -0.0075, -0.0040, -0.0090],
         ...,
         [-0.0206, -0.0134, -0.0168,  ...,  0.0098, -0.0210,  0.0064],
         [-0.0155,  0.0357,  0.0197,  ..., -0.0324, -0.0013,  0.0121],
         [-0.0301, -0.0105,  0.0022,  ..., -0.0111,  0.0353, -0.0132]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0227, -0.0310,  0.0207,  ...,  0.0025,  0.0026, -0.0005],
        requires_grad=True)]

In [9]:
seq_PT.model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50432, 2560)
    (layers): ModuleList(
      (0): GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)
          (act): GELUActivation()
        )
      )
      (1): GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=Tru

In [7]:
prompt = 'Tell me if the following sentence is true or false: immigrants, if well integrated, are an asset for the host country'
out = seq_PT.attribute(
    f'<human>: {prompt}\n <bot>:', 
    step_scores=["probability"],
    # generated_texts = ['LGBT people should have rights', 'LGBT people should have rights'],
    generation_args = {
        'max_new_tokens': 20,
        'min_new_tokens': 4,
        'do_sample': True,
        'temperature': 0.7,
        'top_p': 0.8,
        'top_k': 50,
        'no_repeat_ngram_size': 2,
    },
)
print('Generated text: {output}'.format(output = out.info['generated_texts']))
out.show()

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Attributing with input_x_gradient...: 100%|██████████| 51/51 [00:36<00:00,  1.83s/it]

Generated text: ['<human>: Tell me if the following sentence is true or false: immigrants, if well integrated, are an asset for the host country\n <bot>: The sentence "if well-integrated, immigrants are assets for a country" is false.\n<']


,ĠThe,Ġsentence,"Ġ""",if,Ġwell,-,integrated,",",Ġimmigrants,Ġare,Ġassets,Ġfor,Ġa,Ġcountry,"""",Ġis,Ġfalse,.,Ċ,<
<,0.053,0.05,0.054,0.049,0.037,0.033,0.029,0.028,0.029,0.027,0.027,0.029,0.025,0.027,0.034,0.04,0.046,0.035,0.035,0.043
human,0.06,0.055,0.05,0.044,0.04,0.036,0.032,0.031,0.032,0.028,0.026,0.028,0.027,0.03,0.039,0.044,0.045,0.041,0.039,0.051
>:,0.078,0.077,0.074,0.07,0.06,0.049,0.044,0.044,0.043,0.043,0.04,0.042,0.037,0.039,0.044,0.066,0.061,0.05,0.056,0.068
ĠTell,0.046,0.046,0.042,0.032,0.031,0.028,0.023,0.02,0.02,0.022,0.021,0.016,0.018,0.02,0.022,0.041,0.041,0.024,0.034,0.023
Ġme,0.027,0.028,0.027,0.019,0.019,0.016,0.013,0.013,0.012,0.013,0.012,0.011,0.011,0.011,0.014,0.028,0.023,0.015,0.02,0.012
Ġif,0.036,0.041,0.028,0.024,0.021,0.026,0.014,0.016,0.015,0.017,0.013,0.016,0.014,0.015,0.016,0.032,0.033,0.019,0.024,0.016
Ġthe,0.023,0.021,0.031,0.021,0.016,0.015,0.013,0.019,0.013,0.015,0.013,0.016,0.015,0.012,0.013,0.02,0.019,0.014,0.016,0.018
Ġfollowing,0.034,0.058,0.037,0.031,0.024,0.021,0.019,0.019,0.018,0.018,0.02,0.016,0.016,0.017,0.021,0.024,0.032,0.02,0.023,0.018
Ġsentence,0.063,0.089,0.066,0.062,0.048,0.04,0.034,0.036,0.027,0.034,0.034,0.023,0.029,0.029,0.042,0.046,0.054,0.04,0.039,0.026
Ġis,0.018,0.016,0.013,0.017,0.013,0.011,0.01,0.01,0.01,0.01,0.01,0.009,0.009,0.009,0.009,0.016,0.021,0.012,0.013,0.01


In [8]:
prompt = 'Tell me if the following sentence is true or false: immigrants, if well integrated, are an asset for the host country'
out = seq_RL.attribute(
    f'<human>: {prompt}\n <bot>:', 
    step_scores=["probability"],
    # generated_texts = ['LGBT people should have rights', 'LGBT people should have rights'],
    generation_args = {
        'max_new_tokens': 20,
        'min_new_tokens': 4,
        'do_sample': True,
        'temperature': 0.7,
        'top_p': 0.8,
        'top_k': 50,
        'no_repeat_ngram_size': 2,
    },
)
print('Generated text: {output}'.format(output = out.info['generated_texts']))
out.show()

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Attributing with input_x_gradient...:  61%|██████    | 31/51 [00:00<?, ?it/s]/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.9/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
Attributing with input_x_gradient...: 100%|██████████| 51/51 [00:37<00:00,  1.88s/it]

Generated text: ['<human>: Tell me if the following sentence is true or false: immigrants, if well integrated, are an asset for the host country\n <bot>: The sentence "if well-integrated, immigrants are assets for a country" is false.\n<']


,ĠThe,Ġsentence,"Ġ""",if,Ġwell,-,integrated,",",Ġimmigrants,Ġare,Ġassets,Ġfor,Ġa,Ġcountry,"""",Ġis,Ġfalse,.,Ċ,<
<,0.053,0.05,0.054,0.049,0.037,0.033,0.029,0.028,0.029,0.027,0.027,0.029,0.025,0.027,0.034,0.04,0.046,0.035,0.035,0.043
human,0.06,0.055,0.05,0.044,0.04,0.036,0.032,0.031,0.032,0.028,0.026,0.028,0.027,0.03,0.039,0.044,0.045,0.041,0.039,0.051
>:,0.078,0.077,0.074,0.07,0.06,0.049,0.044,0.044,0.043,0.043,0.04,0.042,0.037,0.039,0.044,0.066,0.061,0.05,0.056,0.068
ĠTell,0.046,0.046,0.042,0.032,0.031,0.028,0.023,0.02,0.02,0.022,0.021,0.016,0.018,0.02,0.022,0.041,0.041,0.024,0.034,0.023
Ġme,0.027,0.028,0.027,0.019,0.019,0.016,0.013,0.013,0.012,0.013,0.012,0.011,0.011,0.011,0.014,0.028,0.023,0.015,0.02,0.012
Ġif,0.036,0.041,0.028,0.024,0.021,0.026,0.014,0.016,0.015,0.017,0.013,0.016,0.014,0.015,0.016,0.032,0.033,0.019,0.024,0.016
Ġthe,0.023,0.021,0.031,0.021,0.016,0.015,0.013,0.019,0.013,0.015,0.013,0.016,0.015,0.012,0.013,0.02,0.019,0.014,0.016,0.018
Ġfollowing,0.034,0.058,0.037,0.031,0.024,0.021,0.019,0.019,0.018,0.018,0.02,0.016,0.016,0.017,0.021,0.024,0.032,0.02,0.023,0.018
Ġsentence,0.063,0.089,0.066,0.062,0.048,0.04,0.034,0.036,0.027,0.034,0.034,0.023,0.029,0.029,0.042,0.046,0.054,0.04,0.039,0.026
Ġis,0.018,0.016,0.013,0.017,0.013,0.011,0.01,0.01,0.01,0.01,0.01,0.009,0.009,0.009,0.009,0.016,0.021,0.012,0.013,0.01


In [6]:
prompt = 'Tell me if the following sentence is true or false: immigrants, if well integrated, are an asset for the host country'
out = seq_FT.attribute(
    f'<human>: {prompt}\n <bot>:', 
    step_scores=["probability"],
    # generated_texts = ['LGBT people should have rights', 'LGBT people should have rights'],
    generation_args = {
        'max_new_tokens': 40,
        'min_new_tokens': 4,
        'do_sample': True,
        'temperature': 0.7,
        'top_p': 0.8,
        'top_k': 50,
        'no_repeat_ngram_size': 2,
    },
)
print('Generated text: {output}'.format(output = out.info['generated_texts']))
out.show()

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Attributing with input_x_gradient...: 100%|██████████| 71/71 [01:21<00:00,  2.05s/it]

Generated text: ['<human>: Tell me if the following sentence is true or false: immigrants, if well integrated, are an asset for the host country\n <bot>: The sentence "if well-integrated, immigrants are assets for a country" is false.\n<humans>: Could you explain why it is not true?\n\nI think it\'s true because']


,ĠThe,Ġsentence,"Ġ""",if,Ġwell,-,integrated,",",Ġimmigrants,Ġare,Ġassets,Ġfor,Ġa,Ġcountry,"""",Ġis,Ġfalse,.,Ċ,<,hum,ans,>:,ĠCould,Ġyou,Ġexplain,Ġwhy,Ġit,Ġis,Ġnot,Ġtrue,?,Ċ,Ċ,I,Ġthink,Ġit,'s,Ġtrue,Ġbecause
<,0.053,0.05,0.054,0.049,0.037,0.033,0.029,0.028,0.029,0.027,0.027,0.029,0.025,0.027,0.034,0.04,0.046,0.035,0.035,0.043,0.084,0.053,0.064,0.039,0.038,0.03,0.026,0.027,0.029,0.031,0.032,0.027,0.03,0.038,0.032,0.034,0.03,0.024,0.023,0.025
human,0.06,0.055,0.05,0.044,0.04,0.036,0.032,0.031,0.032,0.028,0.026,0.028,0.027,0.03,0.039,0.044,0.045,0.041,0.039,0.051,0.137,0.075,0.087,0.053,0.048,0.031,0.029,0.031,0.033,0.033,0.036,0.028,0.033,0.052,0.036,0.039,0.033,0.026,0.027,0.026
>:,0.078,0.077,0.074,0.07,0.06,0.049,0.044,0.044,0.043,0.043,0.04,0.042,0.037,0.039,0.044,0.066,0.061,0.05,0.056,0.068,0.078,0.049,0.111,0.065,0.054,0.048,0.046,0.044,0.041,0.051,0.054,0.04,0.051,0.062,0.053,0.054,0.047,0.04,0.037,0.04
ĠTell,0.046,0.046,0.042,0.032,0.031,0.028,0.023,0.02,0.02,0.022,0.021,0.016,0.018,0.02,0.022,0.041,0.041,0.024,0.034,0.023,0.015,0.011,0.013,0.022,0.023,0.024,0.02,0.019,0.017,0.021,0.024,0.018,0.019,0.017,0.015,0.024,0.018,0.015,0.017,0.016
Ġme,0.027,0.028,0.027,0.019,0.019,0.016,0.013,0.013,0.012,0.013,0.012,0.011,0.011,0.011,0.014,0.028,0.023,0.015,0.02,0.012,0.009,0.006,0.007,0.012,0.011,0.012,0.011,0.012,0.01,0.013,0.014,0.01,0.009,0.009,0.007,0.024,0.014,0.008,0.009,0.01
Ġif,0.036,0.041,0.028,0.024,0.021,0.026,0.014,0.016,0.015,0.017,0.013,0.016,0.014,0.015,0.016,0.032,0.033,0.019,0.024,0.016,0.01,0.006,0.007,0.013,0.011,0.015,0.012,0.013,0.012,0.015,0.019,0.013,0.011,0.009,0.009,0.01,0.011,0.01,0.012,0.01
Ġthe,0.023,0.021,0.031,0.021,0.016,0.015,0.013,0.019,0.013,0.015,0.013,0.016,0.015,0.012,0.013,0.02,0.019,0.014,0.016,0.018,0.009,0.009,0.009,0.009,0.007,0.011,0.012,0.011,0.013,0.014,0.014,0.009,0.009,0.012,0.005,0.01,0.009,0.006,0.012,0.008
Ġfollowing,0.034,0.058,0.037,0.031,0.024,0.021,0.019,0.019,0.018,0.018,0.02,0.016,0.016,0.017,0.021,0.024,0.032,0.02,0.023,0.018,0.012,0.009,0.008,0.013,0.012,0.015,0.013,0.015,0.012,0.02,0.021,0.014,0.011,0.012,0.011,0.012,0.012,0.011,0.013,0.012
Ġsentence,0.063,0.089,0.066,0.062,0.048,0.04,0.034,0.036,0.027,0.034,0.034,0.023,0.029,0.029,0.042,0.046,0.054,0.04,0.039,0.026,0.021,0.012,0.014,0.022,0.019,0.024,0.021,0.024,0.022,0.035,0.037,0.023,0.016,0.019,0.015,0.026,0.024,0.017,0.023,0.026
Ġis,0.018,0.016,0.013,0.017,0.013,0.011,0.01,0.01,0.01,0.01,0.01,0.009,0.009,0.009,0.009,0.016,0.021,0.012,0.013,0.01,0.005,0.005,0.004,0.006,0.005,0.007,0.007,0.009,0.007,0.011,0.014,0.009,0.006,0.005,0.004,0.004,0.006,0.006,0.008,0.006
